# Virtual Try-On: Garment Styling

There are often many different ways a particular garment can be worn - sleeves rolled up or down, waist tucked or
untucked, etc. We refer to this as styling. Nova Canvas supports the following optional styling parameters:

- `longSleeveStyle` - Applies to long-sleeve tops. `"SLEEVE_UP" | "SLEEVE_DOWN"` (Note: only works if the sleeves in the source image are short)
- `tuckingStyle` - Applies to tops that fall below the waist. `"UNTUCKED" | "TUCKED"` (Note: only works if the top in the source image is tucked)
- `outerLayerStyle` - Applies to tops that can open in the front such as jackets. `"CLOSED" | "OPEN"`

_✏️ **NOTE:** An alternate way to control styling is to provide your own mask image which implies a particular styling of the garment._

In the following example, let's style the garment so that the sleeves are rolled up.

<div style="display: flex; gap: 2em; margin-right: 2em">
  <div style="text-align: center; flex: 1">
    <img src="../images/man-upper_body-tshirt.png">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="text-align: center; flex: 1">
    <img src="../images/jersey-long_sleeve.png">
    <strong><tt>referenceImage</tt></strong>
  </div>
   <div style="text-align: center; flex: 1">
    <img src="../images/top_styling_output.png">
    <strong>Example output</strong>
  </div>
</div>


In [ ]:
import logging
import os
from datetime import datetime
from random import randint
from amazon_nova_canvas_utils import (
    generate_images,
    load_image_as_base64,
    base64_to_pil_image,
)
from PIL import Image
from IPython.display import Image

logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_garment_styling_source.jpg"
reference_image_path = "../images/vto-images/vto_garment_styling_reference.jpg"

inference_params = {
    "taskType": "VIRTUAL_TRY_ON",
    "virtualTryOnParams": {
        "sourceImage": load_image_as_base64(source_image_path),
        "referenceImage": load_image_as_base64(reference_image_path),
        "maskType": "GARMENT",
        "garmentBasedMask": {
            "garmentClass": "UPPER_BODY",
            # Add garment styling parameters
            "garmentStyling": {"longSleeveStyle": "SLEEVE_UP"} 
        },
    },
    # The following is optional but provided here for you to experiment with.
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 6.5,
        "seed": randint(0, 2147483646),
    },
}

output_folder = os.path.join("output", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

try:
    response_body = generate_images(
        inference_params=inference_params,
        save_folder_path=output_folder,
        model_id="amazon.nova-canvas-v1:0",
        region_name="us-east-1",
    )

    # An error message may be returned, even if some images were generated.
    if "error" in response_body:
        logging.error(response_body["error"])

    if "images" in response_body:
        # Display all images.
        for image_base64 in response_body["images"]:
            image = base64_to_pil_image(image_base64)
            display(image)

except Exception as e:
    logging.error(e)

print(f"Done! Artifacts saved to {os.path.abspath(output_folder)}")